In [32]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 777.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [33]:
# ============================
# 1. Gerekli Kütüphaneler
# ============================
# Terminal veya Colab için yükleme:
# pip install pdfplumber sentence-transformers langdetect language-tool-python

import pdfplumber
from langdetect import detect
from sentence_transformers import SentenceTransformer, util
import numpy as np

# ============================
# 2. Model Yükleme
# ============================
# Çok dilli (Türkçe+İngilizce) destekli SentenceTransformer modeli
model = SentenceTransformer('all-MiniLM-L6-v2')

# ============================
# 3. Kriterler ve Ağırlıklar
# ============================
criteria_weights = {
    "iş_deneyimi": 25,
    "egitim": 15,
    "teknik_beceriler": 15,
    "ozet": 10,
    "liderlik": 10,
    "sertifikalar": 10,
    "iletisim": 5,
    "portfolyo": 5,
    "diller": 3,
    "referanslar": 2
}

# İdeal açıklamalar (TR & EN)
ideal_texts_tr = {
    "iş_deneyimi": "Önceki pozisyonlar, şirketler, görevler, başarılar ve kariyer gelişimi anlatılmalı.",
    "egitim": "Üniversite, bölüm, not ortalaması, projeler ve burslar hakkında bilgiler.",
    "teknik_beceriler": "Programlama dilleri, yazılım araçları ve sertifikalar açıkça belirtilmeli.",
    "ozet": "Kariyer hedefleri, deneyimler ve güçlü yönler kısa ve öz özetlenmeli.",
    "liderlik": "Liderlik rolleri, organizasyon deneyimleri ve takım çalışması belirtilmeli.",
    "sertifikalar": "Katılım sağlanan kurslar ve alınan sertifikalar listelenmeli.",
    "iletisim": "Telefon, e-posta, LinkedIn gibi iletişim bilgileri bulunmalı.",
    "portfolyo": "GitHub, kişisel web sitesi veya portfolyo linkleri yer almalı.",
    "diller": "Yabancı dil bilgisi ve seviyeleri açıklanmalı.",
    "referanslar": "Referans kişilerin bilgileri veya talep üzerine verileceği belirtilmeli."
}

ideal_texts_en = {
    "iş_deneyimi": "Previous positions, companies, responsibilities, achievements and career progression should be described.",
    "egitim": "Information about university, department, GPA, projects and scholarships.",
    "teknik_beceriler": "Programming languages, software tools and certifications must be clearly stated.",
    "ozet": "Career objectives, experiences and strengths should be summarized briefly.",
    "liderlik": "Leadership roles, organizational experiences and teamwork should be mentioned.",
    "sertifikalar": "Attended courses and obtained certificates should be listed.",
    "iletisim": "Contact information such as phone, email, LinkedIn must be included.",
    "portfolyo": "GitHub, personal website or portfolio links should be provided.",
    "diller": "Foreign language skills and proficiency levels should be explained.",
    "referanslar": "Reference persons' details or statement about references on request."
}

# ============================
# 4. PDF Metin Çıkarma
# ============================
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# ============================
# 5. CV’yi Basit Bölümlere Ayırma
# ============================
def split_cv_into_sections(text, lang="tr"):
    # Basit şekilde anahtar kelimelere göre bölümlere ayırıyoruz
    sections = {}
    lines = text.splitlines()
    current_section = "genel"

    for line in lines:
        clean_line = line.strip().lower()

        if lang == "tr":
            if "iş deneyimi" in clean_line:
                current_section = "iş_deneyimi"
            elif "eğitim" in clean_line:
                current_section = "egitim"
            elif "teknik beceri" in clean_line or "beceri" in clean_line:
                current_section = "teknik_beceriler"
            elif "özet" in clean_line or "hakkımda" in clean_line:
                current_section = "ozet"
            elif "liderlik" in clean_line or "organizasyon" in clean_line:
                current_section = "liderlik"
            elif "sertifika" in clean_line:
                current_section = "sertifikalar"
            elif "iletişim" in clean_line:
                current_section = "iletisim"
            elif "portföy" in clean_line or "github" in clean_line:
                current_section = "portfolyo"
            elif "dil" in clean_line:
                current_section = "diller"
            elif "referans" in clean_line:
                current_section = "referanslar"
        else:  # EN
            if "work experience" in clean_line or "experience" in clean_line:
                current_section = "iş_deneyimi"
            elif "education" in clean_line:
                current_section = "egitim"
            elif "skills" in clean_line:
                current_section = "teknik_beceriler"
            elif "summary" in clean_line or "profile" in clean_line:
                current_section = "ozet"
            elif "leadership" in clean_line:
                current_section = "liderlik"
            elif "certificate" in clean_line:
                current_section = "sertifikalar"
            elif "contact" in clean_line:
                current_section = "iletisim"
            elif "portfolio" in clean_line or "github" in clean_line:
                current_section = "portfolyo"
            elif "language" in clean_line:
                current_section = "diller"
            elif "reference" in clean_line:
                current_section = "referanslar"

        if current_section not in sections:
            sections[current_section] = line + "\n"
        else:
            sections[current_section] += line + "\n"

    return sections

# ============================
# 6. Semantik Benzerlik Skoru
# ============================
def semantic_score(cv_section_text, ideal_text):
    emb1 = model.encode(cv_section_text, convert_to_tensor=True)
    emb2 = model.encode(ideal_text, convert_to_tensor=True)
    cosine_sim = util.pytorch_cos_sim(emb1, emb2).item()
    return cosine_sim  # 0-1 arası

# ============================
# 7. CV Analiz Fonksiyonu
# ============================
def analyze_cv_semantic(cv_text):
    try:
        lang = detect(cv_text)
    except:
        lang = "unknown"

    lang_code = "tr" if lang == "tr" else "en"
    ideal_texts = ideal_texts_tr if lang_code == "tr" else ideal_texts_en

    sections = split_cv_into_sections(cv_text, lang_code)
    total_score = 0
    strengths = []
    weaknesses = []

    for criterion, weight in criteria_weights.items():
        ideal = ideal_texts.get(criterion, "")
        cv_section_text = sections.get(criterion, "")

        if not cv_section_text.strip():
            weaknesses.append(f"{criterion.replace('_', ' ').title()} bölümü eksik.")
            continue

        score = semantic_score(cv_section_text, ideal)
        section_score = score * weight
        total_score += section_score

        if score > 0.5:
            strengths.append(f"{criterion.replace('_', ' ').title()} bölümü iyi (benzerlik: {score:.2f}).")
        else:
            weaknesses.append(f"{criterion.replace('_', ' ').title()} bölümü zayıf (benzerlik: {score:.2f}).")

    total_score = min(100, total_score)  # 100 üzerinden

    return {
        "language": lang,
        "total_score": round(total_score, 2),
        "strengths": strengths,
        "weaknesses": weaknesses
    }

# ============================
# 8. Manuel PDF Yükleme
# ============================
def manual_pdf_upload():
    try:
        from google.colab import files
        print("Lütfen analiz etmek istediğiniz PDF dosyasını seçin:")
        uploaded = files.upload()
        pdf_path = list(uploaded.keys())[0]
        print(f"'{pdf_path}' başarıyla yüklendi.")
        return pdf_path
    except ImportError:
        pdf_path = input("PDF dosyasının tam yolunu girin: ")
        return pdf_path

# ============================
# 9. Ana Program
# ============================
if __name__ == "__main__":
    pdf_path = manual_pdf_upload()
    cv_text = extract_text_from_pdf(pdf_path)
    report = analyze_cv_semantic(cv_text)

    print(f"\nCV Dili: {report['language']}")
    print(f"Toplam Puan: {report['total_score']}/100\n")

    print("Güçlü Yönler:")
    for s in report['strengths']:
        print(f" - {s}")

    print("\nZayıf Yönler:")
    for w in report['weaknesses']:
        print(f" - {w}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Lütfen analiz etmek istediğiniz PDF dosyasını seçin:


Saving Curriculum Vitae - Meriç Özcan.pdf to Curriculum Vitae - Meriç Özcan (6).pdf
'Curriculum Vitae - Meriç Özcan (6).pdf' başarıyla yüklendi.

CV Dili: en
Toplam Puan: 24.38/100

Güçlü Yönler:
 - Portfolyo bölümü iyi (benzerlik: 0.60).

Zayıf Yönler:
 - Iş Deneyimi bölümü zayıf (benzerlik: 0.42).
 - Egitim bölümü zayıf (benzerlik: 0.17).
 - Teknik Beceriler bölümü zayıf (benzerlik: 0.42).
 - Ozet bölümü eksik.
 - Liderlik bölümü eksik.
 - Sertifikalar bölümü eksik.
 - Iletisim bölümü eksik.
 - Diller bölümü zayıf (benzerlik: 0.40).
 - Referanslar bölümü zayıf (benzerlik: 0.42).


In [34]:
# ============================
# 8. Manuel PDF Yükleme
# ============================
def manual_pdf_upload():
    try:
        from google.colab import files
        print("Lütfen analiz etmek istediğiniz PDF dosyasını seçin:")
        uploaded = files.upload()
        pdf_path = list(uploaded.keys())[0]
        print(f"'{pdf_path}' başarıyla yüklendi.")
        return pdf_path
    except ImportError:
        pdf_path = input("PDF dosyasının tam yolunu girin: ")
        return pdf_path

# ============================
# 9. Ana Program
# ============================
if __name__ == "__main__":
    pdf_path = manual_pdf_upload()
    cv_text = extract_text_from_pdf(pdf_path)
    report = analyze_cv_semantic(cv_text)

    print(f"\nCV Dili: {report['language']}")
    print(f"Toplam Puan: {report['total_score']}/100\n")

    print("Güçlü Yönler:")
    for s in report['strengths']:
        print(f" - {s}")

    print("\nZayıf Yönler:")
    for w in report['weaknesses']:
        print(f" - {w}")


Lütfen analiz etmek istediğiniz PDF dosyasını seçin:


Saving Meriç Özcan - Özgeçmiş.pdf to Meriç Özcan - Özgeçmiş (4).pdf
'Meriç Özcan - Özgeçmiş (4).pdf' başarıyla yüklendi.

CV Dili: tr
Toplam Puan: 21.76/100

Güçlü Yönler:
 - Egitim bölümü iyi (benzerlik: 0.53).
 - Iletisim bölümü iyi (benzerlik: 0.54).
 - Portfolyo bölümü iyi (benzerlik: 0.59).

Zayıf Yönler:
 - Iş Deneyimi bölümü eksik.
 - Teknik Beceriler bölümü zayıf (benzerlik: 0.45).
 - Ozet bölümü eksik.
 - Liderlik bölümü eksik.
 - Sertifikalar bölümü eksik.
 - Diller bölümü zayıf (benzerlik: 0.26).
 - Referanslar bölümü zayıf (benzerlik: 0.30).
